In [ ]:
from numba import jit

In [ ]:
import os
import sys
import math
import time
import random
from tkinter import *
from PIL import Image, ImageTk
import numpy as np

In [ ]:
class Mandelbrot():
    def __init__(self, canvasW, canvasH, x=-0.75, y=0, m=1.5, iterations=None, w=None, h=None, zoomFactor=0.01, multi=True):
        self.w, self.h = (round(canvasW*0.9), round(canvasH*0.9)) if None in {w, h} else w, h
        self.iterations = 200 if iterations is None else iterations
        self.xCenter, self.yCenter = x, y
        if canvasW > canvasH:
            self.xDelta = m/(canvasH/canvasW)
            self.yDelta = m
        else:
            self.yDelta = m/(canvasW/canvasH)
            self.xDelta = m
        self.delta = m
        self.multi = multi
        self.xmin = x - self.xDelta
        self.xmax = x + self.xDelta
        self.ymin = y - self.yDelta
        self.ymax = y + self.yDelta
        self.zoomFactor = zoomFactor
        self.yScaleFactor = self.h/canvasH
        self.xScaleFactor = self.w/canvasW
        self.c, self.z = 0, 0

    def shiftView(self, event):
        self.xCenter = translate(event.x*self.xScaleFactor, 0, self.w, self.xmin, self.xmax)
        self.yCenter = translate(event.y*self.yScaleFactor, self.h, 0, self.ymin, self.ymax)
        self.xmax = self.xCenter + self.xDelta
        self.ymax = self.yCenter + self.yDelta
        self.xmin = self.xCenter - self.xDelta
        self.ymin = self.yCenter - self.yDelta

    def zoomOut(self, event):
        self.xCenter = translate(event.x*self.xScaleFactor, 0, self.w, self.xmin, self.xmax)
        self.yCenter = translate(event.y*self.yScaleFactor, self.h, 0, self.ymin, self.ymax)
        self.xDelta /= self.zoomFactor
        self.yDelta /= self.zoomFactor
        self.delta /= self.zoomFactor
        self.xmax = self.xCenter + self.xDelta
        self.ymax = self.yCenter + self.yDelta
        self.xmin = self.xCenter - self.xDelta
        self.ymin = self.yCenter - self.yDelta

    def zoomIn(self, event):
        self.xCenter = translate(event.x*self.xScaleFactor, 0, self.w, self.xmin, self.xmax)
        self.yCenter = translate(event.y*self.yScaleFactor, self.h, 0, self.ymin, self.ymax)
        self.xDelta *= self.zoomFactor
        self.yDelta *= self.zoomFactor
        self.delta *= self.zoomFactor
        self.xmax = self.xCenter + self.xDelta
        self.ymax = self.yCenter + self.yDelta
        self.xmin = self.xCenter - self.xDelta
        self.ymin = self.yCenter - self.yDelta

    def getPixels(self):
        coordinates = []
        #for x in range(self.w):
        #    for y in range(self.h):
        #        coordinates.append((x, y))
        r1 = np.linspace(self.xmin,self.xmax,self.w)
        print(self.w)
        r2 = np.linspace(self.ymin,self.ymax,self.h)
        print(self.h)
        #n3 = np.empty((self.w,self.h))
        print("Using 1 core...")
        pixels = []
        for i in range(self.w):
            for j in range(self.h):
                pixels.append(self.getEscapeTime(r1[i], r2[j]))
        self.pixels = pixels

    def getEscapeTime(self, re, im):
        #re = translate(x, 0, self.w, self.xmin, self.xmax)
        #im = translate(y, 0, self.h, self.ymax, self.ymin)
        #re=cre
        #im=cim
        z, c = complex(re, im), complex(re, im)
        for i in range(1, self.iterations):
            #re2 = re*re
            #im2 = im*im
            #if (re2+im2) > 4:
            if abs(z) > 2:
                return (re, im, i)
            z = z*z + c
            #im = 2*re*im+cim
            #re = re2-im2+cre
        return (re, im, 0)

In [ ]:
class Framework(Frame):
    def __init__(self, parent, h, x=-0.75, y=0, m=1.5, iterations=None, imgWidth=None, imgHeight=None, save=False, multi=True):
        Frame.__init__(self, parent)
        self.parent = parent
        self.parent.title("Mandelbrot")
        self.pack(fill=BOTH, expand=1)
        self.canvas = Canvas(self)

        if None in {imgWidth, imgHeight}:
            imgWidth, imgHeight = h, h
        if imgWidth > imgHeight:
            ratio = imgHeight/imgWidth
            self.canvasW, self.canvasH = h, round(h*ratio)
        else:
            ratio = imgWidth/imgHeight
            self.canvasW, self.canvasH = round(h*ratio), h

        self.fractal = Mandelbrot(self.canvasW, self.canvasH, x=x, y=y, m=m, iterations=iterations, w=imgWidth, h=imgHeight, multi=multi)
        self.setPalette()
        self.pixelColors = []
        self.img = None
        self.save = save
        self.img = Image.new('RGB', (self.fractal.w, self.fractal.h), "black")
        self.draw()

        parent.bind("<Button-1>", self.zoomIn)
        parent.bind("<Button-3>", self.zoomOut)
        parent.bind("<Control-1>", self.shiftView)
        parent.bind("<Control-3>", self.changePalette)
        parent.bind("<Button-2>", self.saveImage)

    def zoomIn(self, event):
        self.fractal.zoomIn(event)
        self.draw()

    def zoomOut(self, event):
        self.fractal.zoomOut(event)
        self.draw()

    def shiftView(self, event):
        self.fractal.shiftView(event)
        self.draw()

    def draw(self):
        print('-' * 20)
        start = time.time()
        self.fractal.getPixels()
        #self.getColors()
        self.drawPixels()
        self.canvas.create_image(0, 0, image=self.background, anchor=NW)
        self.canvas.pack(fill=BOTH, expand=1)
        print("Process took {} seconds".format(round(time.time()-start, 2)))
        print("Current coordinates (x, y, m): {}, {}, {}".format(self.fractal.xCenter, self.fractal.yCenter, self.fractal.delta))

    def setPalette(self):
        palette = [(0, 0, 0)]
        redb = 2 * math.pi / (random.randint(0, 128) + 128)
        redc = 256 * random.random()
        greenb = 2 * math.pi / (random.randint(0, 128) + 128)
        greenc = 256 * random.random()
        blueb = 2 * math.pi / (random.randint(0, 128) + 128)
        bluec = 256 * random.random()
        for i in range(256):
            r = clamp(int(256 * (0.5 * math.sin(redb * i + redc) + 0.5)))
            g = clamp(int(256 * (0.5 * math.sin(greenb * i + greenc) + 0.5)))
            b = clamp(int(256 * (0.5 * math.sin(blueb * i + bluec) + 0.5)))
            palette.append((r, g, b))
        self.palette = palette

    def changePalette(self, event):
        self.setPalette()
        self.pixelColors = []
        self.getColors()
        self.drawPixels()
        self.canvas.create_image(0, 0, image=self.background, anchor=NW)
        self.canvas.pack(fill=BOTH, expand=1)

    def getColors(self):
        pixelColors = []
        for p in self.fractal.pixels:
            pixelColors.append(self.palette[p[2] % 256])
        self.pixelColors = pixelColors

    def drawPixels(self):
        #img = Image.new('RGB', (self.fractal.w, self.fractal.h), "black")
        #pixels = img.load()  # create the pixel map
        
        #for index, p in enumerate(self.fractal.pixels):
        #    pixels[int(p[0]), int(p[1])] = self.palette[p[2] % 256]#self.pixelColors[index]
        img = Image.fromarray(self.colors)
        self.img = img
        if self.save:
            self.saveImage(None)
        photoimg = ImageTk.PhotoImage(img.resize((self.canvasW, self.canvasH)))
        self.background = photoimg

    def saveImage(self, event):
        self.img.save("output/{}.png".format(time.strftime("%Y-%m-%d-%H:%M:%S")), "PNG", optimize=True)


In [ ]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin
    valueScaled = float(value - leftMin) / float(leftSpan)
    return rightMin + (valueScaled * rightSpan)

In [ ]:
def clamp(x):
    return max(0, min(x, 255))

In [ ]:
master = Tk()
height = round(master.winfo_screenheight()*0.3)
render = Framework(master, height)
master.geometry("{}x{}".format(render.canvasW, render.canvasH))
master.mainloop()